In [1]:
import numpy as np
import pandas as pd

from tqdm import tqdm
from commons import *

%matplotlib inline
from matplotlib import pyplot as plt

In [3]:
CSV_TRAIN       = './train.csv'
DATA_DIR        = '../inputs/512/train/'
RESIZE          = 512
IGNORE_CLASSES  = [14]

In [34]:
df_train = pd.read_csv(CSV_TRAIN)
df_train

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,width,height
0,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R16,0.0,0.0,1.0,1.0,2332,2580
1,21a10246a5ec7af151081d0cd6d65dc9,No finding,14,R2,0.0,0.0,1.0,1.0,2954,3159
2,9a5094b2563a1ef3ff50dc5c7ff71345,Aortic enlargement,0,R9,1052.0,715.0,1299.0,966.0,2080,2336
3,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R8,689.0,1313.0,1666.0,1763.0,2080,2336
4,9a5094b2563a1ef3ff50dc5c7ff71345,Pleural thickening,11,R9,1789.0,1729.0,1875.0,1992.0,2080,2336
...,...,...,...,...,...,...,...,...,...,...
30028,7c0ddf911bdb68fca14d7172486827cb,No finding,14,R5,0.0,0.0,1.0,1.0,1994,2430
30029,209e3a5c73c1195d805dd25d086b3c6d,No finding,14,R1,0.0,0.0,1.0,1.0,2336,2836
30030,1bea772246421c37929f8cbc43dba729,No finding,14,R2,0.0,0.0,1.0,1.0,2336,2836
30031,ae86eabab95525b41b8e79883ff1cef9,No finding,14,R7,0.0,0.0,1.0,1.0,2864,3055


# Convert to Coco Dataset

In [35]:
dataset = CocoDataset(df_train.copy(), img_size=512, keep_ratio=False)

In [36]:
dataset.write_all_annotations(save_filename='coco512_nms.json')

100%|██████████| 30033/30033 [00:00<00:00, 498046.96it/s]


## Split .json (COCO Format) into train/val

In [37]:
dataset.train_test_split(in_filename='coco512_nms.json')

Split completed!


FROM SPLITED CSV

In [ ]:
FOLDS_DIR       = '.cache/'
FOLDS           = [0, 1, 2, 3, 4]
TRAIN_FOLDS_DIR = os.path.join(FOLDS_DIR, 'train/')
VAL_FOLDS_DIR   = os.path.join(FOLDS_DIR, 'val/')
EXT             = '.csv'


In [ ]:
for fold in FOLDS:
    train_path  = f'{TRAIN_FOLDS_DIR}{fold}{EXT}'
    val_path    = f'{VAL_FOLDS_DIR}{fold}{EXT}'
    train_df    = pd.read_csv(train_path)
    val_df      = pd.read_csv(val_path)

    dataset = CocoDataset(train_df.copy())
    dataset.write_all_annotations(
            save_filename=f'./coco/{fold}/train.json',
        )
    
    dataset = CocoDataset(val_df.copy())
    dataset.write_all_annotations(
            save_filename=f'./coco/{fold}/val.json',
        )